In [91]:
import glob, re
import pandas as pd
import numpy as np
from sklearn import *
lbl = preprocessing.LabelEncoder()

In [92]:
#"START HKLEE FEATURE"
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('../input/*.csv')}
for k, v in dfs.items(): locals()[k] = v

wkend_holidays = date_info.apply(lambda x: (x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1, axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) 

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=['air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), how='left')['visitors_y'].values

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), on='air_store_id', how='left')['visitors_y'].values

test_visit_var = sample_submission.visitors.map(pd.np.expm1)

data = {
    'tra': pd.read_csv('../input/air_visit_data.csv')
    }

data['tra'] = data['tra'].merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
data['tra'] = data['tra'].merge(visitors, on=['air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = data['tra'].visitors_y.isnull()
data['tra'].loc[missings, 'visitors_y'] = data['tra'][missings].merge(visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), how='left')['visitors_y'].values

missings = data['tra'].visitors_y.isnull()
data['tra'].loc[missings, 'visitors_y'] = data['tra'][missings].merge(visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), on='air_store_id', how='left')['visitors_y'].values

train_visit_var = data['tra'].visitors_y.map(pd.np.expm1)

In [96]:
# read all the data
air_visits = pd.read_csv('../input/air_visit_data.csv')
air_reserve = pd.read_csv('../input/air_reserve.csv')
air_store_info = pd.read_csv('../input/air_store_info.csv')
hol = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
sample_submission = pd.read_csv('../input/sample_submission.csv')

In [97]:
# transfer air_visits
air_visits['visit_date'] = pd.to_datetime(air_visits['visit_date'])
air_visits['dow'] = air_visits['visit_date'].dt.dayofweek
air_visits['year'] = air_visits['visit_date'].dt.year
air_visits['month'] = air_visits['visit_date'].dt.month
air_visits['visit_date'] = air_visits['visit_date'].dt.date

In [98]:
# transfer sample submission
sample_submission['visit_date'] = sample_submission['id'].map(lambda x:str(x).split('_')[2])
sample_submission['air_store_id'] = sample_submission['id'].map(lambda x: '_'.join(str(x).split('_')[:2]))
sample_submission['visit_date'] = pd.to_datetime(sample_submission['visit_date'])
sample_submission['dow'] = sample_submission['visit_date'].dt.dayofweek
sample_submission['year'] = sample_submission['visit_date'].dt.year
sample_submission['month'] = sample_submission['visit_date'].dt.month
sample_submission['visit_date'] = sample_submission['visit_date'].dt.date

In [99]:
unique_stores = sample_submission['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)
stores = pd.merge(stores, air_store_info, how='left', on=['air_store_id']) 

In [100]:
# NEW FEATURES FROM Georgii Vyshnia
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))

In [101]:
for i in range(2):
    stores['air_genre_name'+str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))
for i in range(3):
    stores['air_area_name'+str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ''))

stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

In [102]:
train = pd.merge(air_visits, stores, how='inner', on=['air_store_id','dow']) 
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test = pd.merge(sample_submission, stores, how='inner', on=['air_store_id','dow']) 
test['air_store_id2'] = lbl.transform(test['air_store_id'])

In [103]:
train.drop(['visitors','dow','latitude', 'longitude'], axis=1, inplace=True)
test.drop(['visitors','dow','visit_date','air_store_id', 'latitude', 'longitude'], axis=1, inplace=True)

In [104]:
train['hklee_feature'] = train_visit_var 
test['hklee_feature'] = test_visit_var
train['weight'] = data['tra']['weight']

In [105]:
test.to_csv('test4.csv', index=False)
train.to_csv('train4.csv', index=False)

# Merge with train val

In [106]:
# read all the data
training = pd.read_csv('../train/training_data2.csv')
val = pd.read_csv('../train/validation_data2.csv')
testing = pd.read_csv('../train/submission_with_features2.csv')
train = pd.read_csv('train4.csv')
test = pd.read_csv('test4.csv')

In [107]:
training.drop(['day_of_week','year', 'month', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'Apr', 'Aug',
       'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep', 'air_genre_name', 'air_area_name',
        'Asian', 'Bar Cocktail', 'Cafe Sweets', 'Creative cuisine',
       'Dining bar', 'International cuisine', 'Italian French', 'Izakaya',
       'Japanese food', 'Karaoke Party', 'Okonomiyaki Monja Teppanyaki',
       'Other', 'Western food', 'Yakiniku Korean food', 'Fukuoka-ken',
       'Hiroshima-ken', 'Hokkaidō', 'Hyōgo-ken', 'Miyagi-ken', 'Niigata-ken',
       'Shizuoka-ken', 'Tōkyō-to', 'Ōsaka-fu'], axis=1, inplace=True)


val.drop(['day_of_week','year', 'month', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'Apr', 'Aug',
       'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep', 'air_genre_name', 'air_area_name',
        'Asian', 'Bar Cocktail', 'Cafe Sweets', 'Creative cuisine',
       'Dining bar', 'International cuisine', 'Italian French', 'Izakaya',
       'Japanese food', 'Karaoke Party', 'Okonomiyaki Monja Teppanyaki',
       'Other', 'Western food', 'Yakiniku Korean food', 'Fukuoka-ken',
       'Hiroshima-ken', 'Hokkaidō', 'Hyōgo-ken', 'Miyagi-ken', 'Niigata-ken',
       'Shizuoka-ken', 'Tōkyō-to', 'Ōsaka-fu'], axis=1, inplace=True)

testing.drop(['day_of_week','year', 'month', 'Friday', 'Monday',
       'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'Apr', 'Aug',
       'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep', 'air_genre_name', 'air_area_name',
        'Asian', 'Bar Cocktail', 'Cafe Sweets', 'Creative cuisine',
       'Dining bar', 'International cuisine', 'Italian French', 'Izakaya',
       'Japanese food', 'Karaoke Party', 'Okonomiyaki Monja Teppanyaki',
       'Other', 'Western food', 'Yakiniku Korean food', 'Fukuoka-ken',
       'Hiroshima-ken', 'Hokkaidō', 'Hyōgo-ken', 'Miyagi-ken', 'Niigata-ken',
       'Shizuoka-ken', 'Tōkyō-to', 'Ōsaka-fu'], axis=1, inplace=True)

In [108]:
training = pd.merge(training, train, how='left', on=['air_store_id', 'visit_date'])
val = pd.merge(val, train, how='left', on=['air_store_id', 'visit_date'])
testing = pd.merge(testing, test, how='left', on=['id'])

In [109]:
training.to_csv('training_data3.csv', index=False)
val.to_csv('validation_data3.csv', index=False)

In [110]:
testing.to_csv('submission_with_features3.csv', index=False)